In [ ]:
# Install Packages
!pip install sentence_transformers
!pip install nltk

In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from Korpora import Korpora

In [2]:
corpus = Korpora.load('open_subtitles')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : TRAC (https://trac.edgewall.org/)
    Repository : http://opus.nlpl.eu/OpenSubtitles-v2018.php
    References :
        - P. Lison and J. Tiedemann, 2016, OpenSubtitles2016: Extracting Large Parallel Corpora
          from Movie and TV Subtitles. In Proceedings of the 10th International Conference on
          Language Resources and Evaluation (LREC 2016)

    This is a new collection of translated movie subtitles from http://www.opensubtitles.org/.

    [[ IMPORTANT ]]
    If you use the OpenSubtitle corpus: Please, add a link to http://www.opensubtitles.org/
    to your website and to your reports and publications produced with the data!
    I promised this when I got the data from the providers of that website!

 

In [3]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")

driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [5]:
class google_translation:
    def __init__(self, input_text):
        import urllib.parse

        # 한글 퍼센트 인코딩
        self.url_input = urllib.parse.quote(input_text.encode('utf8'))

        # 키워드를 포함하는 url
        self.url = f'https://translate.google.co.kr/?hl=ko&sl=ko&tl=en&text={self.url_input}&op=translate'

        # 번역 결과를 가리키는 xpath
        self.xpath = '/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[3]/c-wiz[2]/div[8]/div/div[1]/span[1]/span/span'
    
    def get_translation(self):

        # 웹페이지 on
        driver.get(self.url)

        # 웹페이지 로딩까지 최대 10초 대기
        driver.implicitly_wait(10)

        # 웹페이지에서 해당 위치의 값 불러와 저장
        output_text_list = driver.find_elements(By.XPATH, self.xpath)

        # 텍스트만 반환
        return output_text_list[0].text

class papago_translation(google_translation):
    def __init__(self, input_text):
        import urllib

        self.url_input = urllib.parse.quote(input_text)
        self.url = f'https://papago.naver.com/?sk=ko&tk=en&st={self.url_input}'
        self.xpath = '/html/body/div/div/div[1]/section/div/div[1]/div[2]/div/div[5]/div/span'

In [15]:
from tqdm.notebook import tqdm
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()

data_dict = {'subtitle_kor': [],
             'subtitle_eng': [], 
             'trans_google': [], 
             'trans_papago': [], 
             'sim_google': [], 
             'sim_papago': [], 
             'unigram_overlap_google': [], 
             'unigram_overlap_papago': [], 
             'BLEU2_google': [], 
             'BLEU2_papago': []}

# Data we use
start_index = 2500
end_index = 5000
whole_data = list(zip(corpus.get_all_texts(), corpus.get_all_pairs()))

for kor_txt, eng_txt in tqdm(whole_data[start_index:end_index]):
    subtitle_kor = kor_txt
    
    # lists of sentences
    trans_google = google_translation(kor_txt).get_translation()
    trans_papago = papago_translation(kor_txt).get_translation()
    subtitle_eng = eng_txt

    # Compute embedding for lists
    embeddings_google = model.encode([trans_google], convert_to_tensor=True)
    embeddings_papago = model.encode([trans_papago], convert_to_tensor=True)
    embeddings_Target = model.encode([subtitle_eng], convert_to_tensor=True)

    # Compute cosine-similarits
    sim_google = util.cos_sim(embeddings_google, embeddings_Target).item()
    sim_papago = util.cos_sim(embeddings_papago, embeddings_Target).item()
    
    # Unigram overlap
    tokenized_google = [word.lower() for word in tokenizer.tokenize(trans_google.replace('-', ' ')) if word not in '.,:']
    tokenized_papago = [word.lower() for word in tokenizer.tokenize(trans_papago.replace('-', ' ')) if word not in '.,:']
    tokenized_target = [word.lower() for word in tokenizer.tokenize(subtitle_eng.replace('-', ' ')) if word not in '.,:']
    
    unigram_overlap_google = sentence_bleu([tokenized_target], tokenized_google, weights=(1,))
    unigram_overlap_papago = sentence_bleu([tokenized_target], tokenized_papago, weights=(1,))
    
    # 2-gram overlap with smoothing
    BLEU2_google = sentence_bleu([tokenized_target], tokenized_google, weights=(.5,.5), smoothing_function=SmoothingFunction().method1)
    BLEU2_papago = sentence_bleu([tokenized_target], tokenized_papago, weights=(.5,.5), smoothing_function=SmoothingFunction().method1)
    
    # Construct Data Dictionary
    for key in data_dict:
        data_dict[key].append(eval(f'{key}'))

  0%|          | 0/2500 [00:00<?, ?it/s]

IndexError: list index out of range

In [8]:
data_for_eda = pd.DataFrame(data_dict)
data_for_eda

,subtitle_kor,subtitle_eng,trans_google,trans_papago,sim_google,sim_papago,unigram_overlap_google,unigram_overlap_papago,BLEU2_google,BLEU2_papago
0,"폭설이 내리고 우박, 진눈깨비가 퍼부어도 눈보라가 몰아쳐도 강풍이 불고 비바람이 휘...","Through the snow and sleet and hail, through t...",Even if the snowfall falls and the snow and sl...,"Heavy snow, hail, sleet, blizzard, strong wind...",0.476623,0.718432,0.317082,0.063503,0.217699,0.008001
1,우리의 한결같은 심부름꾼 황새 아저씨 가는 길을 그 누가 막으랴!,"ever faithful, ever true, nothing stops him, h...",Who will stop the way to our constant errands.,"Who will stop our constant errand boy, Mr. Stork?",0.222570,0.229132,0.000000,0.000000,0.000000,0.000000
2,황새 아저씨를 기다리세요,Look out for Mr Stork That persevering chap,Wait for the stork,Wait for Mr. Stork,0.600879,0.705570,0.183940,0.183940,0.047493,0.047493
3,찾아와 선물을 주실 거예요,He'll come along and drop a bundle in your lap,You will come and give you a gift,They'll come and give me a present,0.411141,0.367302,0.257733,0.343645,0.050304,0.183686
4,가난하든 부자이든 상관이 없답니다,You may be poor or rich It doesn't matter which,It doesn't matter whether it's poor or rich,It doesn't matter if you're poor or rich,0.732840,0.813101,0.633386,0.723870,0.564265,0.603225
...,...,...,...,...,...,...,...,...,...,...
2495,개가 아니라 우리가 말하는 나입니다.,"It's me we're talking about, not dogs.","It's not a dog, but I say.","It's not the dog, it's me we're talking about.",0.705835,0.898236,0.330936,0.666667,0.204258,0.603023
2496,오늘 내가 어떻게 보는 것이 중요합니다.,It's important how I look today.,It is important to see how I see it today.,It's important how I look today.,0.705796,1.000000,0.500000,1.000000,0.235702,1.000000
2497,나는 결혼 할거야.,I'm gonna get married.,I'm going to get married.,I'm gonna get married.,0.948955,1.000000,0.666667,1.000000,0.516398,1.000000
2498,결혼 했니?,"Married, you?",Are you married?,Are you married?,0.817232,0.817232,0.750000,0.750000,0.158114,0.158114


In [14]:
data_for_eda.to_csv('Score_Data.csv',index=False, encoding='utf-8-sig')